In [151]:
from numpy import *
from math import log2
from pandas import *

In [152]:
class Node:
    def __init__(self, value=None, trueBranch=None, falseBranch=None, results=None, col=-1, summary=None, data=None):
        self.value = value
        self.trueBranch = trueBranch
        self.falseBranch = falseBranch
        self.results = results
        self.col = col
        self.summary = summary
        self.data = data
    def print(self):
        print("value: ", self.value)
        print("truBrach: ", self.trueBranch)
        print("falseBranch: ", self.falseBranch)
        print("results: ", self.results)
        print("col: ", self.col)
        print("summary: ", self.summary)
        print("data: ", self.data)


In [153]:
def loadData(filename):
    df = read_csv(filename)
    dataMat = mat(df)
    dataList = []
    for i in range(dataMat.shape[0]):
        dataList.append(dataMat[i].tolist()[0])
    return dataList
dataList = [
    [1, 1, 'yes'],
    [1, 1, 'yes'],
    [1, 0, 'no'],
    [0, 1, 'no'],
    [0, 1, 'no']
]
dataList = loadData('Titanic_new.csv')

In [154]:
def calcCount(datas):
    # 将输入的数据汇总(input dataSet)
    results = {}
    for data in datas:
        # data[-1] means dataType
        if data[-1] not in results:
            results[data[-1]] = 1
        else:
            results[data[-1]] += 1
    return results

In [155]:
def gini(List):
    length = len(List)
    results = calcCount(List)
    imp = 0.0
    for i in results:
        imp += (results[i] / length) ** 2
    return 1 - imp

In [156]:
def splitDatas(rows, value, column):
    # 根据条件分离数据集(splitDatas by value,column)
    list1 = []
    list2 = []
    if isinstance(value, float):  # for float type
        for row in rows:
            if (row[column] <= value):
                list1.append(row)
            else:
                list2.append(row)
    else:  # for String type or int
        for row in rows:
            if row[column] == value:
                list1.append(row)
            else:
                list2.append(row)

    return (list1, list2)

In [157]:
def build(rows, minLen = 3):
    # 递归建立决策树,当gain < 0 时停止递归
    currentGain = gini(rows)
    column_length = len(rows[0])
    rows_length = len(rows)
    # 预剪枝
    # if rows_length < minLen or column_length <= 1: return calcCount(rows) 
    best_gain = 0.0; best_value = None; best_set = None

    # choose the best gain
    for col in range(column_length - 1):
        col_value_set = set([x[col] for x in rows])
        for value in col_value_set:
            list1, list2 = splitDatas(rows, value, col)
            p = len(list1) / rows_length
            gain = currentGain - p * gini(list1) - (1 - p) * gini(list2)
            if gain > best_gain:
                best_gain = gain
                best_value = (col, value)
                best_set = (list1, list2)

    dcY = {'impurity': currentGain, 'samples': rows_length}

    # stop or not stop
    if best_gain > 0:
        trueBranch = build(best_set[0])
        falseBranch = build(best_set[1])
        return Node(col=best_value[0], value=best_value[1], trueBranch=trueBranch, falseBranch=falseBranch, summary=dcY)
    else: # 预剪枝
        return Node(results=calcCount(rows), summary=dcY, data=rows)
Tree = build(dataList)

In [158]:
def classify(data, tree):
    if tree.results != None:
        return tree.results
    else:
        branch = None
        v = data[tree.col]
        if isinstance(v, float):
            if v <= tree.value:
                branch = tree.trueBranch
            else:
                branch = tree.falseBranch
        else:
            if v == tree.value:
                branch = tree.trueBranch
            else:
                branch = tree.falseBranch
        return classify(data, branch)

In [159]:
def prune(tree, miniGain, evaluationFunction=gini):
    # 剪枝, when gain < mini Gain，合并(merge the trueBranch and the falseBranch)

    if tree.trueBranch.results == None: prune(tree.trueBranch, miniGain, evaluationFunction)
    if tree.falseBranch.results == None: prune(tree.falseBranch, miniGain, evaluationFunction)

    if tree.trueBranch.results != None and tree.falseBranch.results != None:
        len1 = len(tree.trueBranch.data)
        len2 = len(tree.falseBranch.data)
        len3 = len(tree.trueBranch.data + tree.falseBranch.data)
        p = float(len1) / (len1 + len2)
        gain = gini(tree.trueBranch.data + tree.falseBranch.data) - p * gini(tree.trueBranch.data) - (1 - p) * gini(tree.falseBranch.data)
        if (gain < tree):
            tree.data = tree.trueBranch.data + tree.falseBranch.data
            tree.results = calcCount(tree.data)
            tree.trueBranch = None
            tree.falseBranch = None

In [160]:
import pandas as pd
df = read_csv('test.csv')
df['Age'].fillna(df['Age'].mean(), inplace = True)
df['Fare'].fillna(df['Fare'].mean(), inplace = True)
fw = df['PassengerId']
df = df.drop(['PassengerId'], axis=1)
n = len(df)
train = []
for j in range(n):
    L = []
    for i in range(7):
        L.append(df.iloc[j, i])
    train.append(L)
ans = []
for i in range(n):
    result = classify(train[i], Tree)
    if result == None: assert(0)
    sortedResult = sorted(result.items(), key=lambda x:x[1], reverse=True)
    ans.append(sortedResult[0][0])

data = [[fw[i], ans[i]] for i in range(n)]
fw = pd.DataFrame(data, columns=['PassengerId', 'Survived'])
fw.to_csv('submission.csv', index=False)
